## Annotation bootstrapping
- via a cheap dictionary model; though any model can be used that implements the `predict_single` method

In [ ]:
from clear_bow.classifier import DictionaryClassifier

dictionaries = {
    "customer_service": ["customer service", "service", "experience"],
    "pricing": ["expensive", "cheap", "dear", "dollars", "cents"],
    "billing": ["quarterly", "online", "phone"],
    "product": [
        "quality",
        "product",
        "superior",
        "inferior",
        "fast",
        "efficient",
        "range",
        "selection",
        "replaced",
    ],
    "competitor": ["another provider", "competitor", "leaving", "will not return"],
}
dc = DictionaryClassifier(classifier_type="multi_label", label_dictionary=dictionaries)


In [ ]:
# verify model predictions for n examples, optionally tweak how candidate examples are retrieved
text_col = "text"
label_col = "label"

stateful_annotations = annotate_n_examples_per_class(
    dc,
    df,
    text_col,
    n_examples=2,
    prediction_thresh=0.7,
    rank_candidates=True,
    max_candidates=2,
    label_col=label_col,
)


In [ ]:
# # re-iterate annotations on the same dataset (pre-existing experiment?), updating as needed
# concat = pd.concat([(df
#                     .query('new_text not in @stateful_annotations.new_text')
#                     ),
#                    stateful_annotations])
# stateful_annotations = annotate_n_examples_per_class(dc, concat, text_col, n_examples=10, prediction_thresh=0.75, rank_candidates=True, max_candidates=20)


In [ ]:
# if multi-label, backfill across the label space
stateful_annotations = backfill_multi_label_objects(
    dc, stateful_annotations, text_col, label_col, prediction_thresh=0.7
)


## Flair zero/few-shot learning
- Format tiny datset as an experimernt, train, eval etc.

In [ ]:
# quantize annotations into usual format; some data loss here (explicit rejections => "-1" values)
quantized_annotations = stateful_annotations.copy(deep=True)
quantized_annotations[label_col] = quantized_annotations[label_col].apply(
    threshold_one_hot_dictionary
)
quantized_annotations[label_col] = normalize_label_space(
    quantized_annotations[label_col].tolist()
)

quantized_annotations = (
    quantized_annotations
    # ensure each record has at least one record
    .reset_index(drop=True).pipe(lambda x: x[contains_one_label(x, label_col)])
)


In [ ]:
# format annotations, create experiment with usual structure
df, label_cols = one_hot_encode_multi_label_df(quantized_annotations, label_col)
train_dev_test_splits = create_multi_label_train_dev_test_splits(
    df, text_col=text_col, label_cols=label_cols, split_size=0.4, label_dict=True
)  # larger test portions RE: low support

In [ ]:
experiment_output_dir = Path("/Users/samhardyhey/Desktop/tars_doodle")
create_experiment_output_dir(experiment_output_dir, train_dev_test_splits)


In [ ]:
res = compare_experiments_barplot(
    experiment_paths=[experiment_output_dir],
    title="TARS eval.",
)